In [40]:
import openai
import numpy as np
import pandas as pd
import requests
import os

from numpy.linalg import norm

In [18]:
api_key = os.environ['OPENAI_API_KEY']
openai.api_key = api_key

In [19]:
data = 'Homo sapiens HepG2 genetically modified (insertion) using CRISPR targeting H. sapiens PIN1'

In [20]:
model = 'text-embedding-ada-002'

In [21]:
response = openai.Embedding.create(
    input=data,
    model=model,
)

In [28]:
search_embedding = np.array(response['data'][0]['embedding'])

In [29]:
response = openai.Embedding.create(
    input='chip-seq experiments in humans',
    model=model,
)

In [30]:
query_embedding = np.array(response['data'][0]['embedding'])

In [31]:
query_embedding

array([-0.02876394,  0.00399126,  0.00338419, ..., -0.02887126,
       -0.01529426,  0.0010498 ])

In [34]:
search_embedding.dot(query_embedding)

0.8424903302990354

In [42]:
np.dot(query_embedding, search_embedding) / norm(search_embedding) * norm(query_embedding)

0.8424903587550906